# Notebook for interacting with the $(n,n'\gamma)$ $\textit{"Baghdad Atlas"}$

For more information please refer to the reference articles [1978De41, 2021Hu05].

Aaron Hurst <amhurst@berkeley.edu>

In [ ]:
import sqlite3

In [ ]:
#Determine Python version
import sys
db = None
major_version = sys.version_info[0]
minor_version = sys.version_info[1]
micro_version = sys.version_info[2]
if major_version == 2: db = str('atlas_baghdad_py2.db')
elif major_version == 3: db = str('atlas_baghdad_py3.db')
print("Running Python {0}.{1}.{2}".format(major_version,minor_version,micro_version))

In [ ]:
# Simple query: extract all information from 'sample' table for elements with Z < 15
conn = sqlite3.connect(db)
curs = conn.cursor()
sql_cmd = "SELECT * FROM sample WHERE Z < 30;"
curs.execute(sql_cmd)
db_info = curs.fetchall()
for entry in db_info:
    print(entry)
conn.commit()
conn.close()

In [ ]:
# Populate a list with names of enriched isotopes; count number of enriched isotopes
conn = sqlite3.connect(db)
curs = conn.cursor()
sql_cmd = "SELECT element FROM sample \
    WHERE A != 0;"
curs.execute(sql_cmd)
db_info = curs.fetchall()
conn.close()

num_elements = 0
for entry in db_info: num_elements += 1
print(db_info)
print("Number of enriched isotopes = {0}".format(int(num_elements)))

In [ ]:
# Retrieve gamma-ray energy and intensity data for Fe (Z=26) and output query to file

Z = 28 #Define atomic number for query
element = str('Nickel') #Define a name for the file

conn = sqlite3.connect(db)
curs = conn.cursor()

sql_cmd = "SELECT energy_gamma, d_energy_gamma, intensity_gamma, d_intensity_gamma FROM nucleus WHERE nuc_Z = {0};"\
    .format(int(Z))
curs.execute(sql_cmd)
db_info = curs.fetchall()

width = 8
with open('data_{0}.dat'.format(element), mode='wt') as f:
    for i,entry in enumerate(db_info):
        energy, d_energy = entry[0], entry[1]
        intensity, d_intensity = entry[2], entry[3] 
        f.write("{0: <{width}}\t{1: <{width}}\t{2: <{width}}\t{3: <{width}}\n"
                .format(energy, d_energy, intensity, d_intensity, width=width))
        if i<20: print(entry)
    f.close()
                    
conn.commit()
conn.close()

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_context("poster")
sns.set("talk")

In [ ]:
from numpy import loadtxt, array

In [ ]:
%matplotlib notebook
#Retrieve energy and intensity data from SQL database and calculate cross sections.
#Display cross-section data as bar plot and save to file.
#Adjust Z, Chem_symb, and A below to change nucleus.

Z = int(75) # Define atomic number
Chem_symb = str('Re') # Define corresponding chemical symbol, e.g. 'W' for natural W; '186W' for 186-enriched W
A = int(0) # A=0 for natural sample, otherwise use mass number of enriched isotope

enriched = None
if A == 0: enriched = str('N')
else: enriched = str('E')

conn = sqlite3.connect(db)
curs = conn.cursor()
sql_cmd = "SELECT energy_gamma, ((nucleus.intensity_gamma/100.0) * sample.N * (143.0/100.0)) \
    AS cross_section FROM nucleus, sample \
    WHERE nuc_Z = {0} \
    AND nucleus.nuc_Z = sample.Z \
    AND nucleus.sample = '{1}' AND nucleus.nuc_symb = '{2}' \
    AND nucleus.energy_gamma <= 1000.0;".format(Z,enriched,Chem_symb)
# Just do an Ig (relative BR) plot instead
#sql_cmd = "SELECT energy_gamma, (nucleus.intensity_gamma/100.0) * sample.N \
#    AS intensity_gamma FROM nucleus, sample \
#    WHERE nuc_Z = {0} \
#    AND nucleus.nuc_Z = sample.Z \
#    AND nucleus.sample = '{1}' AND nucleus.nuc_symb = '{2}' \
#    AND nucleus.transition_type != 't' AND nucleus.residual = '35Cl' AND nucleus.residual_type = 'f'\
#    AND nucleus.energy_gamma <= 5000.0;".format(Z,enriched,Chem_symb)
curs.execute(sql_cmd)
db_info = array(curs.fetchall())
conn.close()

f, ax = plt.subplots(figsize=(9,6))
#ax.bar(db_info[:,0],db_info[:,1],label=r"$\sigma_{\gamma}$(%s)"%str(Chem_symb),color='black',linewidth=3,
ax.bar(db_info[:,0],db_info[:,1],label=r"$I_{\gamma}$(%s)"%str(Chem_symb),color='black',linewidth=3,
      edgecolor='black')
ax.legend(loc=1)
ax.set_yscale('log')
ax.set_xlabel(r'$E_{\gamma}$ [keV]')
#ax.set_ylabel(r'$\sigma_{\gamma}$ [mb]')
ax.set_ylabel(r'$I_{\gamma}$')
#ax.xaxis.set_ticks(range(0,2001,500))
f.suptitle(r"{0}$(n,n'\gamma)$".format(Chem_symb))

plt.show()
f.savefig("{0}_cs_query.pdf".format(Chem_symb),dpi=f.dpi)
f.savefig("{0}_cs_query.png".format(Chem_symb),dpi=f.dpi)

In [ ]:
from math import sqrt
class CrossSection(object):
    FeCS = 143.0 
    FeRatio = 1.430 
    d_FeCS = 29.0 
    d_FeRatio = 0.290 
    #FeCS = 586.0 # FRM-II
    #FeRatio = 5.860 # FRM-II
    #d_FeCS = 41.0 # FRM-II
    #d_FeRatio = 0.410
    RI = 100.0
    def __init__(self,N,dN):
        self.N, self.dN = N, dN
    def cross_section(self,Ig,dIg):
        return ((Ig/CrossSection.RI)*self.N*CrossSection.FeRatio)
    def d_cross_section(self,cs,Ig,dIg):
        return (cs*sqrt((dIg/Ig)**2+(self.dN/self.N)**2+(CrossSection.d_FeRatio/CrossSection.FeRatio)**2))
    def d_cross_section_reduced(self,cs):
        return (cs*sqrt((dN/N)**2+(CrossSection.d_FeRatio/CrossSection.FeRatio)**2))

In [ ]:
%matplotlib notebook
#Extract energy and intensity information from SQL database and store in list.
#Calculate cross sections from list data using above "CrossSection" class.
#Write results to file and then plot data from file; save plot to file.
#Adjust Z, Chem_symb, and A below to change nucleus.
import re

Z = int(75) # Define atomic number
Chem_symb = str('Re') # Define corresponding chemical symbol, e.g. 'W' for natural W; '186W' for 186-enriched W
A = int(0) # A=0 for natural sample, otherwise use mass number of enriched isotope

enriched = None
if A == 0: enriched = str('N')
else: enriched = str('E')

conn = sqlite3.connect(db)
curs = conn.cursor()

N, dN = None, None
sql_cmd = "SELECT sample.N, sample.dN from sample \
    WHERE sample.Z = {0} AND sample.A = {1};".format(Z,A)
curs.execute(sql_cmd)
db_info = list(curs.fetchall())
for NdN in db_info:
    N, dN = NdN[0], NdN[1]
    
print(N,dN)
CS = CrossSection(float(N),float(dN))

sql_cmd = "SELECT nucleus.energy_gamma, nucleus.d_energy_gamma, nucleus.intensity_gamma, nucleus.d_intensity_gamma, \
    nucleus.residual \
    from nucleus \
    WHERE nuc_Z = {0}\
    AND nucleus.sample = '{1}' AND nucleus.nuc_symb = '{2}'\
    AND nucleus.transition_type != 't' AND nucleus.residual_type = 'f'\
    AND nucleus.residual LIKE '%' || nucleus.nuc_symb || '%';".format(Z,enriched,Chem_symb)
# Specify condition on residual nucleus, e.g., 185Re: "AND nucleus.residual = '185Re'"
curs.execute(sql_cmd)
db_info = curs.fetchall()
conn.close()

#new_db_info = list()
width = 10
with open("{0}_cs_results.dat".format(Chem_symb), mode='wt') as wf:
    for row in db_info: 
        energy_gamma, d_energy_gamma = row[0], row[1]
        intensity_gamma, d_intensity_gamma = float(row[2]), float(row[3])
        cs, d_cs = None, None
        try:
            cs = CS.cross_section(intensity_gamma,d_intensity_gamma)
            d_cs = CS.d_cross_section(cs,intensity_gamma,d_intensity_gamma)
        except ZeroDivisionError:
            d_cs = CS.d_cross_section_reduced(cs)
        cs, d_cs = "%.3f"%float(cs), "%.3f"%float(d_cs)
        #print(energy_gamma,d_energy_gamma,cs,d_cs)
        wf.write("{0: <{width}}\t{1: <{width}}\t{2: <{width}}\t{3: <{width}}\n"
                 .format(energy_gamma, d_energy_gamma, cs, d_cs, width=width))
        #new_db_info.append([float(energy_gamma),float(d_energy_gamma),float(cs),float(d_cs)])
    wf.close()

#xnew_db_info = array(new_db_info)
#print(xnew_db_info)

f, ax = plt.subplots(figsize=(9,6))

data = loadtxt("{0}_cs_results.dat".format(Chem_symb))
energy_gamma, cs, d_cs = list(), list(), list()
try:
    energy_gamma = data[:,0]
    cs = data[:,2]
    d_cs = data[:,3]
except IndexError:
    energy_gamma = data[0]
    cs = data[2]
    d_cs = data[3]

Chem_label = None
if bool(re.match("[0-9]",Chem_symb))==True:
    isotope_mass = str(A)
    isotope_label = str(re.findall("[A-Z]+[a-z]?",Chem_symb)[0])
    Chem_label = str(r'$^{%s}$%s'%(isotope_mass,isotope_label))
else: Chem_label = Chem_symb

ax.errorbar(energy_gamma, cs, yerr=d_cs, label=r"$\sigma_{\gamma}$(%s)"%str(Chem_label),color='blue', \
            marker='o',markersize=5,capsize=5,ecolor='blue',mec='blue',mew=2,elinewidth=2,linestyle='None')
#ax.errorbar(energy_gamma, cs, yerr=d_cs, label=r"$\sigma_{\gamma}$(%s) Baghdad Atlas"%str(Chem_label),color='blue', \
#            marker='o',markersize=5,capsize=5,ecolor='blue',mec='blue',mew=2,elinewidth=2,linestyle='None')


ax.legend(loc=1,frameon=True,shadow=True,fancybox=True)
ax.set_yscale('log')
ax.set_xlabel(r'$E_{\gamma}$ [keV]')
ax.set_ylabel(r'$\sigma_{\gamma}$ [mb]')
f.suptitle(r"{0}$(n,n'\gamma)$".format(Chem_label))

plt.show()
f.savefig("{0}_cs_dcs_query.pdf".format(Chem_symb),dpi=f.dpi)
f.savefig("{0}_cs_dcs_query.png".format(Chem_symb),dpi=f.dpi,transparent=False)

In [ ]:
%matplotlib notebook
# Plot branching ratios (i.e. raw gamma-ray intensities) with error bars for tungsten data
# Adjust Z, Chem_symb, and A below to change nucleus.

Z = int(74)# Define atomic number
Chem_symb = str('W') # Define corresponding chemical symbol, e.g. 'W' for natural W; '186W' for 186-enriched W
A = int(0) # A=0 for natural sample, otherwise use mass number of enriched isotope

enriched = None
if A == 0: enriched = str('N')
else: enriched = str('E')

conn = sqlite3.connect(db)
curs = conn.cursor()
sql_cmd = "SELECT energy_gamma, d_energy_gamma, intensity_gamma, d_intensity_gamma \
    AS cross_section FROM nucleus \
    WHERE nuc_Z = {0} \
    AND nucleus.sample = '{1}' AND nucleus.nuc_symb = '{2}';".format(Z,enriched,Chem_symb)
curs.execute(sql_cmd)
db_info = array(curs.fetchall())
conn.close()

f, ax = plt.subplots(figsize=(9,6))

ax.errorbar(db_info[:,0],db_info[:,2],yerr=db_info[:,3],label=r"$I_{\gamma}$(%s)"%str(Chem_symb),color='gold', \
            marker='o',markersize=5,capsize=5,ecolor='blue',mew=2,elinewidth=2,linestyle='None')

ax.legend(loc=1)
ax.set_yscale('log')
ax.set_xlabel(r'$E_{\gamma}$ [keV]')
ax.set_ylabel(r'$I_{\gamma}$')
f.suptitle(r"{0}$(n,n'\gamma)$".format(Chem_symb))

plt.show()
f.savefig("{0}_Ig_query.pdf".format(Chem_symb),dpi=f.dpi)
f.savefig("{0}_Ig_query.png".format(Chem_symb),dpi=f.dpi)

# References

[1978De41]: A.M. Demidov, L.I. Govor, Yu.K. Cherepantsev, M.R. Ahmed, S. Al-Najjar, M.A. Al-Amili, N. Al-Assafi, N. Rammo, "Atlas of Gamma-Ray Spectra from the Inelastic Scattering of Reactor Fast Neutrons", Nuclear Research Institute, Baghdad, Iraq (Moscow, Atomizdat 1978).

[2021Hu05]: A.M. Hurst, L.A. Bernstein, T. Kawano, A.M. Lewis, K. Song, "The Baghdad Atlas: A relational database of inelastic neutron-scattering ($n,n'\gamma$) data", Nucl. Instrum. Methods Phys. Res. Sect. A 995, 165095 (2021).